# Processing MIMIC-IV Data: ICD Codes and Notes

This notebook processes MIMIC-IV datasets, specifically discharge notes and ICD diagnosis/procedure codes, to prepare them for downstream analysis.

## Data Download
- Downloads the required datasets using `wget` with user credentials:
  - `diagnoses_icd.csv.gz`
  - `procedures_icd.csv.gz`
  - `discharge.csv.gz`
  - extra: `d_icd_diagnoses.csv.gz` and `d_icd_procedures.csv.gz`

## Data Parsing Functions
- Functions to clean and reformat ICD codes:
  - `reformat_icd9cm_code`
  - `reformat_icd9pcs_code`
  - `reformat_icd10cm_code`
- Utility functions for data processing:
  - `parse_code_dataframe` for grouping and aggregating codes by patient ID.
  - `parse_notes_dataframe` for filtering and deduplicating clinical notes.
  - `remove_rare_codes` to remove rarely occurring codes below a specified threshold.

## Preprocessing Steps
1. **Load Data**: Import discharge notes, diagnosis codes, and procedure codes.
2. **Reformat data**:
   - Rename columns for consistency.
   - Format code types rows to include version and procedure vs. diagnosis code differentiation
   - Standardize and format ICD codes with punctuation (e.g., `0019` → `001.9`).
   - Process diagnoses and procedure datasets by removing duplicates and Nans, and aggregating ICD_codes for each admission id and wrap them as a list
   - Processes the discharge notes by removing notes with missing text and filtering out duplicate records
3. **Aggregate Data**:
   - Merge diagnosis and procedure datasets.
   - Join with discharge notes.
   - Output: Full MIMIC-IV dataset
4. **Filter ICD Versions**:
   - Create separate datasets for ICD-10 and ICD-9 codes.
5. **ICD Datasets Cleanup**:
   - Drop unnecessary columns.
   - Exclude ICD codes appearing less than a specified number of times.
   - Remove rows with no codes.
   - Deduplicate patient records.
   - Added step: Remove note ids that are present in both ICD-9 and ICD-10 dataset
6. **Extra**: Loading, processing and saving dictionary datasets

## Outputs
- Saves processed data as Parquet files:
  - Full dataset: `mimiciv.parquet`
  - Filtered ICD-10 dataset: `mimiciv_icd10.parquet`
  - Filtered ICD-9 dataset: `mimiciv_icd9.parquet`

## Key Libraries Used
- `polars`: For high-performance DataFrame operations.
- `logging`: For tracking processing steps and outputs.

This preprocessing pipeline ensures clean and standardized datasets, ready for tasks such as ICD code prediction or text classification. 

## 0. Downloading Required Data
- The code uses the `wget` command to download datasets from the PhysioNet MIMIC-IV repository.
- Three files are downloaded:
  1. **diagnoses_icd.csv.gz**: Contains ICD diagnosis codes for patient admissions.
  2. **procedures_icd.csv.gz**: Contains ICD procedure codes for patient admissions.
  3. **discharge.csv.gz**: Contains discharge summary notes for patient admissions.
  4. **Extra**: `d_icd_diagnoses.csv.gz` and `d_icd_procedures.csv.gz`
- The `wget` command uses authentication (`--user` and `--password`) to access the secured datasets.

In [ ]:
username = "yourusername"
password = "yourpassword"

!mkdir -p raw_data

!wget -r -N -c -np --user {username} --password {password} -P raw_data https://physionet.org/files/mimiciv/2.2/hosp/diagnoses_icd.csv.gz
!wget -r -N -c -np --user {username} --password {password} -P raw_data https://physionet.org/files/mimiciv/2.2/hosp/procedures_icd.csv.gz
!wget -r -N -c -np --user {username} --password {password} -P raw_data https://physionet.org/files/mimic-iv-note/2.2/note/discharge.csv.gz
!wget -r -N -c -np --user {username} --password {password} -P raw_data https://physionet.org/files/mimiciv/2.2/hosp/d_icd_diagnoses.csv.gz
!wget -r -N -c -np --user {username} --password {password} -P raw_data https://physionet.org/files/mimiciv/2.2/hosp/d_icd_procedures.csv.gz

## 1. Import libraries, Set Standard Column Names, & Define Utility Functions

- **Setting Up Logging and Input/Output Paths**:
    - Logging is configured to provide status updates and detailed information about each processing step.
    - Logs are saved in a standardized format that includes timestamps, log levels, and messages.
    - Raw data files (input) are labelled
    - Output saved to PATH(data/processed_data)

- **Code Formatting Functions**:
  - These functions (`reformat_icd9cm_code`, `reformat_icd9pcs_code`, `reformat_icd10cm_code`) add proper punctuation to ICD-9-CM, ICD-9-PCS, and ICD-10-CM codes, respectively.
  - This ensures all codes adhere to standardized formatting for further processing.

- **`parse_code_dataframe`**:
  - Takes a dataframe containing codes and organizes it into a structured format:
    - Removes duplicates and null values.
    - Groups codes by `hadm_id` (hospital admission ID) and code type.
    - Returns a dataframe with grouped codes as lists.

- **`parse_notes_dataframe`**:
  - Processes the discharge notes:
    - Removes notes with missing text.
    - Filters out duplicate records.

- **`remove_rare_codes`**:
  - Filters out ICD codes that appear less than a specified number of times.
  - Provides detailed statistics:
    - The number of deleted codes.
    - Affected IDs and changes in the number of codes for these IDs.


In [22]:
import logging
import random
from pathlib import Path
import polars as pl

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# Load input files and process the data
input_filepath = Path("raw_data/physionet.org/files")
output_filepath = Path("processed_data")
output_filepath.mkdir(parents=True, exist_ok=True)

ID_COLUMN = "_id"
SUBJECT_ID_COLUMN = "subject_id"
TEXT_COLUMN = "text"

def reformat_icd9cm_code(code: str) -> str:
    """
    Adds a punctuation in a ICD-9-CM code that is without punctuations.
    Before: 0019
    After:  001.9
    """
    if "." in code:
        return code

    if code.startswith("E"):
        if len(code) > 4:
            return code[:4] + "." + code[4:]
    elif len(code) > 3:
        return code[:3] + "." + code[3:]
    return code

def reformat_icd9pcs_code(code: str) -> str:
    """
    Adds a punctuation in a ICD-9-PCS code that is without punctuations.

    Before: 0019
    After:  00.19
    """

    if "." in code:
        return code
    if len(code) > 2:
        return code[:2] + "." + code[2:]
    return code

def reformat_icd10cm_code(code: str) -> str:
    """
    Adds a punctuation in a ICD-10-CM code that is without punctuations.
    Before: A019
    After:  A01.9
    """
    if len(code) > 3:
        return code[:3] + "." + code[3:]
    else:
        return code

def parse_code_dataframe(
    df: pl.DataFrame,
    code_column: str = "diagnosis_codes",
    code_type_column: str = "diagnosis_code_type",
) -> pl.DataFrame:
    """Change names of colums, remove duplicates and Nans, and takes a dataframe and a column name
    and returns a series with the column name and a list of codes.

    Example:
        Input:
                subject_id  _id     target
                       2   163353     V3001
                       2   163353      V053
                       2   163353      V290

        Output:
            target    [V053, V290, V3001]

    Args:
        row (pd.DataFrame): Dataframe with a column of codes.
        col (str): column name of the codes.

    Returns:
        pd.Series: Series with the column name and a list of codes.
    """

    df = df.filter(df[code_column].is_not_null())
    df = df.unique(subset=[ID_COLUMN, code_column])
    df = df.group_by([ID_COLUMN, code_type_column]).agg(
        pl.col(code_column).map_elements(list, return_dtype=pl.List(pl.Utf8)).alias(code_column)
    )
    return df


def parse_notes_dataframe(df: pl.DataFrame) -> pl.DataFrame:
    """Parse the notes dataframe by filtering out notes with no text and removing duplicates."""
    df = df.filter(df[TEXT_COLUMN].is_not_null())
    df = df.unique(subset=[ID_COLUMN, TEXT_COLUMN])
    return df

def remove_rare_codes(
    df: pl.DataFrame, code_columns: list[str], min_count: int
) -> pl.DataFrame:
    """Removes codes that appear less than min_count times in the dataframe,
    and prints the number of deleted codes, affected IDs, and total codes for affected IDs 
    (before and after filtering) for each column.

    Args:
        df (pl.DataFrame): DataFrame with codes.
        code_columns (list[str]): List of columns with codes.
        min_count (int): Minimum number of times a code must appear in the DataFrame to be kept.

    Returns:
        pl.DataFrame: DataFrame with codes that appear more than min_count times.
    """
    total_ids = len(df["_id"].unique())

    for code_column in code_columns:
        # Explode the codes into rows for counting
        code_exploded = df[["_id", code_column]].explode(code_column)
        original_count = len(code_exploded)

        # Count occurrences of each code
        code_counts = code_exploded[code_column].value_counts()
        codes_to_include = set(
            code_counts.filter(code_counts["count"] >= min_count)[code_column]
        )

        # Filter codes based on the threshold
        code_exploded_filtered = code_exploded.filter(
            pl.col(code_column).is_in(codes_to_include)
        )
        filtered_count = len(code_exploded_filtered)

        # Calculate how many codes were deleted
        deleted_codes = original_count - filtered_count

        # Identify affected IDs
        affected_exploded = code_exploded.join(
            code_exploded_filtered, on=["_id", code_column], how="anti"
        )
        affected_ids = len(affected_exploded["_id"].unique())

        # Total codes in affected IDs before and after filtering
        total_codes_before = (
            code_exploded.filter(pl.col("_id").is_in(affected_exploded["_id"].unique()))
            .shape[0]
        )
        total_codes_after = (
            code_exploded_filtered.filter(
                pl.col("_id").is_in(affected_exploded["_id"].unique())
            ).shape[0]
        )

        print(
            f"Column '{code_column}':"
            f"\n  Deleted {deleted_codes} codes from {affected_ids} IDs "
            f"(out of {total_ids} total IDs)."
            f"\n  Total codes in affected IDs: {total_codes_before} before, {total_codes_after} after."
        )

        # Regroup the filtered codes and re-join with the original DataFrame
        code_filtered = code_exploded_filtered.group_by("_id").agg(pl.col(code_column))
        df = df.drop(code_column)
        df = df.join(code_filtered, on="_id", how="left")

    return df

random.seed(10)

## 2. Loading, Formatting, and Processing Dataframes
  - Load the data from input filepath
  - Formatting
      - Columns in the dataframes are renamed to maintain consistency
      - Formatting code type columns to include version and procedure vs. diagnosis code differentiation
      - Format the diagnosis and procedures codes by adding punctuation points
  - Process the codes and the notes
      - Run `parse_code_dataframe` function to: remove duplicates and Nans, and aggregate ICD_codes for each admission id and wrap them as a list
      - Run `parse_notes_dataframe` which processes the discharge notes by removing notes with missing text and filtering out duplicate records
      - Merge diagnosis and procedure datasets
      - Join with discharge notes
      - Output: Full MIMIC-IV dataset

In [23]:
# Load the dataframes
mimic_notes = pl.read_csv(
    input_filepath / "mimic-iv-note/2.2/note/discharge.csv.gz"
)
mimic_diag = pl.read_csv(
    input_filepath / "mimiciv/2.2/hosp/diagnoses_icd.csv.gz",
    schema_overrides={"icd_code": str},
)
mimic_proc = pl.read_csv(
    input_filepath / "mimiciv/2.2/hosp/procedures_icd.csv.gz",
    schema_overrides={"icd_code": str},
)

In [12]:
# Rename the columns
mimic_notes = mimic_notes.rename(
    {
        "hadm_id": ID_COLUMN,
        "subject_id": SUBJECT_ID_COLUMN,
        "text": TEXT_COLUMN,
    }
)
mimic_diag = mimic_diag.rename(
    {
        "hadm_id": ID_COLUMN,
        "icd_code": "diagnosis_codes",
        "icd_version": "diagnosis_code_type",
    }
).drop(["subject_id"])
mimic_proc = mimic_proc.rename(
    {
        "hadm_id": ID_COLUMN,
        "icd_code": "procedure_codes",
        "icd_version": "procedure_code_type",
    }
).drop(["subject_id"])

In [13]:
# Format the code type columns
mimic_diag = mimic_diag.with_columns(
    mimic_diag["diagnosis_code_type"].cast(pl.Utf8)
)
mimic_diag = mimic_diag.with_columns(
    mimic_diag["diagnosis_code_type"].str.replace("10", "icd10cm")
)
mimic_diag = mimic_diag.with_columns(
    mimic_diag["diagnosis_code_type"].str.replace("9", "icd9cm")
)

mimic_proc = mimic_proc.with_columns(
    mimic_proc["procedure_code_type"].cast(pl.Utf8)
)
mimic_proc = mimic_proc.with_columns(
    mimic_proc["procedure_code_type"].str.replace("10", "icd10pcs")
)
mimic_proc = mimic_proc.with_columns(
    mimic_proc["procedure_code_type"].str.replace("9", "icd9pcs")
)

In [14]:
# Format the diagnosis codes by adding punctuation points
formatted_codes = (
    pl.when(mimic_diag["diagnosis_code_type"] == "icd10cm")
    .then(mimic_diag["diagnosis_codes"].map_elements(reformat_icd10cm_code, return_dtype=pl.Utf8))
    .otherwise(mimic_diag["diagnosis_codes"].map_elements(reformat_icd9cm_code, return_dtype=pl.Utf8))
)
mimic_diag = mimic_diag.with_columns(formatted_codes)

# Format the procedure codes by adding punctuation points
formatted_codes = (
    pl.when(mimic_proc["procedure_code_type"] == "icd10pcs")
    .then(mimic_proc["procedure_codes"])
    .otherwise(mimic_proc["procedure_codes"].map_elements(reformat_icd9pcs_code, return_dtype=pl.Utf8))
)
mimic_proc = mimic_proc.with_columns(formatted_codes)

In [15]:
# Process codes and notes
mimic_diag = parse_code_dataframe(
    mimic_diag,
    code_column="diagnosis_codes",
    code_type_column="diagnosis_code_type",
)
mimic_proc = parse_code_dataframe(
    mimic_proc,
    code_column="procedure_codes",
    code_type_column="procedure_code_type",
)

mimic_notes = parse_notes_dataframe(mimic_notes)
mimic_codes = mimic_diag.join(mimic_proc, on=ID_COLUMN, how='full', coalesce=True)
mimiciv = mimic_notes.join(mimic_codes, on=ID_COLUMN, how="inner")
mimiciv = mimiciv.with_columns(
    mimiciv["note_type"].str.replace("DS", "discharge_summary")
)

## 3. Filtering and Splitting ICD Codes
- Splits the combined data into two subsets:
  - **ICD-10 Data**: Filters rows with `icd10cm` and `icd10pcs` codes.
  - **ICD-9 Data**: Filters rows with `icd9cm` and `icd9pcs` codes.
- Post-split data processing:
    - Rare codes are removed using the `remove_rare_codes` function:
        - Deletes codes that appear less than 10 times.
        - Logs details about removed codes and affected IDs.
    - Duplicates and rows with no codes are dropped.
    - Added step: Remove note ids that are present in both ICD-9 and ICD-10 dataset

In [6]:
#mimiciv = pl.read_parquet('processed_data/mimiciv.parquet')
# remove not used columns
data = mimiciv.drop(["note_seq", "charttime", "storetime"])

In [7]:
# only keep ICD-10 codes
data_icd10 = data.filter(
    (pl.col("diagnosis_code_type") == "icd10cm")
    | (pl.col("procedure_code_type") == "icd10pcs")
)

# only keep ICD-9 codes
data_icd9 = data.filter(
    (pl.col("diagnosis_code_type") == "icd9cm")
    | (pl.col("procedure_code_type") == "icd9pcs")
)

In [8]:
# remove rare codes
data_icd10 = remove_rare_codes(data_icd10, ["diagnosis_codes", "procedure_codes"], 10)
data_icd9 = remove_rare_codes(data_icd9, ["diagnosis_codes", "procedure_codes"], 10)

Column 'diagnosis_codes':
  Deleted 28838 codes from 21248 IDs (out of 122316 total IDs).
  Total codes in affected IDs: 347038 before, 318200 after.
Column 'procedure_codes':
  Deleted 73910 codes from 67413 IDs (out of 122316 total IDs).
  Total codes in affected IDs: 114707 before, 40797 after.
Column 'diagnosis_codes':
  Deleted 13023 codes from 11371 IDs (out of 209359 total IDs).
  Total codes in affected IDs: 152072 before, 139049 after.
Column 'procedure_codes':
  Deleted 86480 codes from 86079 IDs (out of 209359 total IDs).
  Total codes in affected IDs: 99311 before, 12831 after.


In [20]:
row_count = data_icd10.shape[0]
print(f"The ICD-10 DataFrame has {row_count} rows.")

row_count = data_icd9.shape[0]
print(f"The ICD-9 DataFrame has {row_count} rows.")

The ICD-10 DataFrame has 122319 rows.
The ICD-9 DataFrame has 209361 rows.


In [10]:
# drop duplicates
data_icd10 = data_icd10.unique(subset=["_id"])
data_icd9 = data_icd9.unique(subset=["_id"])

In [11]:
row_count = data_icd10.shape[0]
print(f"The ICD-10 DataFrame has {row_count} rows.")

row_count = data_icd9.shape[0]
print(f"The ICD-9 DataFrame has {row_count} rows.")

The ICD-10 DataFrame has 122316 rows.
The ICD-9 DataFrame has 209359 rows.


In [12]:
# filter out rows with no codes
data_icd10 = data_icd10.filter(
    pl.col("diagnosis_codes").is_not_null() | pl.col("procedure_codes").is_not_null()
)

data_icd9 = data_icd9.filter(
    pl.col("diagnosis_codes").is_not_null() | pl.col("procedure_codes").is_not_null()
)

In [14]:
row_count = data_icd10.shape[0]
print(f"The ICD-10 DataFrame has {row_count} rows.")

row_count = data_icd9.shape[0]
print(f"The ICD-9 DataFrame has {row_count} rows.")

The ICD-10 DataFrame has 122278 rows.
The ICD-9 DataFrame has 209326 rows.


### Added step: Remove note ids that are present in both ICD-9 and ICD-10 dataset

In [15]:
def count_and_remove_matching_note_ids(icd10_df: pl.DataFrame, icd9_df: pl.DataFrame):
    # Get unique note_ids from both DataFrames
    icd10_note_ids = set(icd10_df["note_id"].to_list())
    icd9_note_ids = set(icd9_df["note_id"].to_list())
    
    # Find matching note_ids
    matching_note_ids = icd10_note_ids.intersection(icd9_note_ids)
    
    # Filter out matching note_ids
    icd10_df_filtered = icd10_df.filter(~icd10_df["note_id"].is_in(matching_note_ids))
    icd9_df_filtered = icd9_df.filter(~icd9_df["note_id"].is_in(matching_note_ids))
    
    return icd10_df_filtered, icd9_df_filtered

data_icd10, data_icd9= count_and_remove_matching_note_ids(data_icd10, data_icd9)

In [16]:
row_count = data_icd10.shape[0]
print(f"The ICD-10 DataFrame has {row_count} rows.")

row_count = data_icd9.shape[0]
print(f"The ICD-9 DataFrame has {row_count} rows.")

The ICD-10 DataFrame has 122271 rows.
The ICD-9 DataFrame has 209319 rows.


## 4. Saving the Filtered Data
- Saves the filtered datasets (`mimiciv.parquet`, `mimiciv_icd10.parquet`, and `mimiciv_icd9.parquet`) for downstream analysis.
- Logs the locations of the saved files.

In [28]:
# Save the processed data
output_mimic = output_filepath / "mimiciv.parquet"
mimiciv.write_parquet(output_mimic)
logger.info(f"Processed data saved to {output_mimic}")

2025-01-30 08:56:21,373 - __main__ - INFO - Processed data saved to processed_data/mimiciv.parquet


In [20]:
# Save the processed data
output_icd10 = output_filepath / "mimiciv_icd10.parquet"
data_icd10.write_parquet(output_icd10)
logger.info(f"Processed data saved to {output_icd10}")

output_icd9 = output_filepath / "mimiciv_icd9.parquet"
data_icd9.write_parquet(output_icd9)
logger.info(f"Processed data saved to {output_icd9}")

2025-01-30 09:22:33,353 - __main__ - INFO - Processed data saved to processed_data/mimiciv_icd10.parquet
2025-01-30 09:22:48,003 - __main__ - INFO - Processed data saved to processed_data/mimiciv_icd9.parquet


## 5. Extra: Loading, processing and saving dictionary datasets

In [ ]:
dict_diag = pl.read_csv(
    input_filepath / "mimiciv/2.2/hosp/d_icd_diagnoses.csv.gz",
    schema_overrides={"icd_code": str},
)
dict_proc = pl.read_csv(
    input_filepath / "mimiciv/2.2/hosp/d_icd_procedures.csv.gz",
    schema_overrides={"icd_code": str},
)

In [5]:
# Format the code type columns
dict_diag = dict_diag.with_columns(
    dict_diag["icd_version"].cast(pl.Utf8)
)
dict_diag = dict_diag.with_columns(
    dict_diag["icd_version"].str.replace("10", "icd10cm")
)
dict_diag = dict_diag.with_columns(
    dict_diag["icd_version"].str.replace("9", "icd9cm")
)

dict_proc = dict_proc.with_columns(
    dict_proc["icd_version"].cast(pl.Utf8)
)
dict_proc = dict_proc.with_columns(
    dict_proc["icd_version"].str.replace("10", "icd10pcs")
)
dict_proc = dict_proc.with_columns(
    dict_proc["icd_version"].str.replace("9", "icd9pcs")
)

In [6]:
# Format the diagnosis codes by adding punctuation points
formatted_codes = (
    pl.when(dict_diag["icd_version"] == "icd10cm")
    .then(dict_diag["icd_code"].map_elements(reformat_icd10cm_code, return_dtype=pl.Utf8))
    .otherwise(dict_diag["icd_code"].map_elements(reformat_icd9cm_code, return_dtype=pl.Utf8))
)
dict_diag = dict_diag.with_columns(formatted_codes)

# Format the procedure codes by adding punctuation points
formatted_codes = (
    pl.when(dict_proc["icd_version"] == "icd10pcs")
    .then(dict_proc["icd_code"])
    .otherwise(dict_proc["icd_code"].map_elements(reformat_icd9pcs_code, return_dtype=pl.Utf8))
)
dict_proc = dict_proc.with_columns(formatted_codes)

In [149]:
# Merge diagnosis and procedures ICD code dictionaries
dict_icd = dict_diag.vstack(dict_proc)

In [157]:
# Save the processed data
output_dict = output_filepath / "dict_icd.parquet"
dict_icd.write_parquet(output_dict)
logger.info(f"Processed data saved to {output_dict}")

2025-01-29 09:54:27,280 - __main__ - INFO - Processed data saved to data/processed_data/dict_icd.parquet


In [11]:
icd10cm_dict = dict_diag.filter(dict_diag["icd_version"] == "icd10cm")

# Save the processed data
output_dict = output_filepath / "icd10cm_dict.parquet"
icd10cm_dict.write_parquet(output_dict)
logger.info(f"Processed data saved to {output_dict}")

2025-02-03 09:21:26,207 - __main__ - INFO - Processed data saved to processed_data/icd10cm_dict.parquet


In [12]:
icd10cm_dict

icd_code,icd_version,long_title
str,str,str
"""A00""","""icd10cm""","""Cholera"""
"""A00.0""","""icd10cm""","""Cholera due to Vibrio cholerae…"
"""A00.1""","""icd10cm""","""Cholera due to Vibrio cholerae…"
"""A00.9""","""icd10cm""","""Cholera, unspecified"""
"""A01""","""icd10cm""","""Typhoid and paratyphoid fevers"""
…,…,…
"""Z99.2""","""icd10cm""","""Dependence on renal dialysis"""
"""Z99.3""","""icd10cm""","""Dependence on wheelchair"""
"""Z99.8""","""icd10cm""","""Dependence on other enabling m…"
